In [81]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
import numpy as np
from sklearn.metrics import classification_report
# import keras and tensorflow
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# import stop word list
from nltk.corpus import stopwords
from sklearn.metrics import precision_recall_fscore_support

In [82]:
# reading training data
train = pd.read_json('train.json',orient='column')
print train.shape

(1335, 3)


In [83]:
# print parameters
train.columns.values

array([u'category', u'content', u'headline'], dtype=object)

In [84]:
# reading test data
test = pd.read_json('dev.json',orient='column')
print test.shape

(445, 3)


In [85]:
def preprocess_data( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [86]:
clean_review = preprocess_data( train["content"][0] )
print clean_review

flanker colin charvis unlikely play part wales final two games six nations charvis missed three wales victories ankle injury recovery slower expected figure scotland game thought unlikely ready final game said wales physio mark davies sonny parker continuing struggle neck injury hal luscombe fit murrayfield trip centre parker slim chance involved scots march luscombe return fitness missing france match hamstring trouble timely boost said wales assistant coach scott johnson positive hal hope raring go comes back mix adds depth gives us options replacement hooker robin mcbryde remains doubt picking knee ligament damage paris last saturday getting reviewed know end week robin looking added johnson hopeful early say stage steve jones dragons likely drafted mcbryde fails recover


In [87]:
# get all the contents into a file or list
num_contents = train["content"].size
contents_train = []
for i in xrange( 0, num_contents ):
    contents_train.append( preprocess_data( train["content"][i] ) )

In [88]:
print "Cleaning and parsing the training data"
clean_train_contents = []
for i in xrange( 0, num_contents ):
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_contents )                                                                    
    clean_train_contents.append( preprocess_data( train["content"][i] ))
print len(clean_train_contents)

Cleaning and parsing the training data
Review 1000 of 1335

1335


In [89]:
print "Creating the bag of words\n"
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 300) 

train_features = vectorizer.fit_transform(clean_train_contents)

#converting the result to array
train_features = train_features.toarray()

print len(train_features)
no_of_words = []
for i in range(0,len(train_features)):
    no_of_words.append(len(clean_train_contents[i].split(' ')))

# averaging the bag of words features based on number of words in each content
train_features_avg = []
for i in range(0,len(train_features)):
    train_features_avg.append(train_features[i]/float(no_of_words[i]))

print len(train_features_avg)

Creating the bag of words

1335
1335


In [90]:
# preparing test data
# Get the number of reviews based on the dataframe column size
num_test_contents = test["content"].size

print num_test_contents
# Initialize an empty list to hold the clean reviews
clean_test_contents = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_test_contents ):
    clean_test_contents.append( preprocess_data( test["content"][i] ) )
print len(clean_test_contents)

445
445


In [91]:
# extracting features for test data
test_features = vectorizer.fit_transform(clean_test_contents)

#converting the result to array
test_features = test_features.toarray()

print len(test_features)
no_of_words = []
for i in range(0,len(test_features)):
    no_of_words.append(len(clean_test_contents[i].split(' ')))

#print no_of_words

# averaging the bag of words features based on number of words in each content
test_features_avg = []
for i in range(0,len(test_features)):
    test_features_avg.append(test_features[i]/float(no_of_words[i]))

#print test_features_avg

445


In [92]:
# processing labels
print train["category"].unique()
objdict = dict()
objIndex = 1

for objtype in train["category"].unique():
    if not objdict.has_key(objtype):
        objdict[objtype] = objIndex
        objIndex = objIndex + 1
        print objtype,objdict[objtype]
    train[["category"] == objtype] = objdict[objtype]
    
y_train = np.zeros((1335,5))
for i in xrange(0, 1335):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    value = objdict[train["category"][i]]
    y_train[i][value-1] = 1
#print labels
print y_train.shape

# creating test labels
y_test = np.zeros((445,5))
y_target = np.zeros((445,1))
for i in xrange(0,445):
    value = objdict[test["category"][i]]
    y_test[i][value-1] = 1
    y_target[i] = value -1
print y_test.shape, y_target.shape

[u'sport' u'business' u'entertainment' u'tech' u'politics']
sport 1
business 2
entertainment 3
tech 4
politics 5
(1335, 5)
(445, 5) (445, 1)


In [93]:
# Parameters
batch_size = 128
input_size = 300
num_classes = 5
epochs = 300

In [94]:
# the data, shuffled and split between train and test sets
x_train = train_features_avg
x_test = test_features_avg
x_train = np.array(x_train).astype('float32')
x_test = np.array(x_test).astype('float32')

In [ ]:
# defining model an training
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(512, activation='relu', input_dim=input_size))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=10000,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=445)

print score

Epoch 1/10000
1335/1335 [==============================] - 0s - loss: 1.6083 - acc: 0.2187     
Epoch 2/10000
1335/1335 [==============================] - 0s - loss: 1.6030 - acc: 0.2449     
Epoch 3/10000
1335/1335 [==============================] - 0s - loss: 1.5998 - acc: 0.2449     
Epoch 4/10000
1335/1335 [==============================] - 0s - loss: 1.5983 - acc: 0.2449     
Epoch 5/10000
1335/1335 [==============================] - 0s - loss: 1.5971 - acc: 0.2449     
Epoch 6/10000
1335/1335 [=========

In [98]:
# printing other metrics
prediction = model.predict(x_test,batch_size=445)
print prediction.shape[0]

y_pred = np.zeros((prediction.shape[0],1))
for i in range(0,prediction.shape[0]):
    y_pred[i] = np.argmax(prediction[i])

target_names = ['sport', 'business', 'entertainment','tech','politics']
print(classification_report(y_pred, y_target, target_names=target_names))

445
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [